In [1]:
from xtquant import xtdata
import datetime
import time
import logging
import numpy as np
import backtest as bt
from backtest import Context
import pandas as pd
import random
from xtquant.xttrader import XtQuantTrader

In [2]:
current_date = datetime.datetime.now()
c_date_str = current_date.strftime("%Y%m%d")
logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levelname)s - %(message)s', filename = c_date_str + '.txt')


In [4]:
def trader_connect():
    # miniQMT安装路径
    mini_qmt_path = r'D:\huanglong\QuantTrading\国金QMT交易端模拟\bin.x64/../userdata_mini'
    # 创建session_id
    session_id = int(random.randint(100000, 999999))
    # 创建交易对象
    xt_trader = XtQuantTrader(mini_qmt_path, session_id)
    # 启动交易对象
    xt_trader.start()
    # 连接客户端
    connect_result = xt_trader.connect()
    print(connect_result)
    if connect_result == 0:
        print('连接成功')
    from xtquant.xttype import StockAccount
    # QMT账号
    account_id = '55004948'
    # 创建账号对象
    account = StockAccount(account_id)
    # 订阅账号
    res = xt_trader.subscribe(account)
    print(res)
    asset = xt_trader.query_stock_asset(account)
    print('账号类型：', asset.account_type)
    print('资金账号：', asset.account_id)
    print('可用金额：', asset.cash)
    print('冻结金额：', asset.frozen_cash)
    print('持仓市值：', asset.market_value)
    print('总资产：', asset.total_asset)
    return connect_result, account

In [5]:
connect_result, account = trader_connect()

0
连接成功
0
账号类型： 2
资金账号： 55004948
可用金额： 16247902.93
冻结金额： 0.0
持仓市值： 3861704.4
总资产： 20109689.33


In [6]:
#coding:utf-8
import time, datetime, traceback, sys
from xtquant import xtdata
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant

#定义一个类 创建类的实例 作为状态的容器
class _a():
    pass
A = _a()
A.bought_list = []
A.hsa = xtdata.get_stock_list_in_sector('沪深A股')

def interact():
    """执行后进入repl模式"""
    import code
    code.InteractiveConsole(locals=globals()).interact()
xtdata.download_sector_data()

def f(data):
    now = datetime.datetime.now()
    for stock in data:
        if stock not in A.hsa:
            continue
        cuurent_price = data[stock]['lastPrice']
        pre_price = data[stock]['lastClose']
        ratio = cuurent_price / pre_price - 1 if pre_price > 0 else 0
        if ratio > 0.09 and stock not in A.bought_list:
            print(f"{now} 最新价 买入 {stock} 200股")
            async_seq = xt_trader.order_stock_async(acc, stock, xtconstant.STOCK_BUY, 200, xtconstant.LATEST_PRICE, -1, 'strategy_name', stock)
            A.bought_list.append(stock)
    
class MyXtQuantTraderCallback(XtQuantTraderCallback):
    import logging
    current_date = datetime.datetime.now()
    c_date_str = current_date.strftime("%Y%m%d")
    logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levelname)s - %(message)s', filename = c_date_str + '.txt')
    def on_disconnected(self):
        """
        连接断开
        :return:
        """
        print(datetime.datetime.now(),'连接断开回调')
        connect_result, account = trader_connect()
        while connect_result != 0:
            connect_result = trader_connect()

    def on_stock_order(self, order):
        """
        委托回报推送
        :param order: XtOrder对象
        :return:
        """
        print(datetime.datetime.now(), '委托回调', order.order_remark)
        logging.info('委托回调')


    def on_stock_trade(self, trade):
        """
        成交变动推送
        :param trade: XtTrade对象
        :return:
        """
        print(datetime.datetime.now(), '成交回调', trade.order_remark)
        logging.info('成交回调')


    def on_order_error(self, order_error):
        """
        委托失败推送
        :param order_error:XtOrderError 对象
        :return:
        """
        # print("on order_error callback")
        # print(order_error.order_id, order_error.error_id, order_error.error_msg)
        print(f"委托报错回调 {order_error.order_remark} {order_error.error_msg}")
        logging.info('委托报错回调')

    def on_cancel_error(self, cancel_error):
        """
        撤单失败推送
        :param cancel_error: XtCancelError 对象
        :return:
        """
        print(datetime.datetime.now(), sys._getframe().f_code.co_name)

    def on_order_stock_async_response(self, response):
        """
        异步下单回报推送
        :param response: XtOrderResponse 对象
        :return:
        """
        print(f"异步委托回调 {response.order_remark}")
        logging.info('异步委托回调')

    def on_cancel_order_stock_async_response(self, response):
        """
        :param response: XtCancelOrderResponse 对象
        :return:
        """
        print(datetime.datetime.now(), sys._getframe().f_code.co_name)

    def on_account_status(self, status):
        """
        :param response: XtAccountStatus 对象
        :return:
        """
        print(datetime.datetime.now(), sys._getframe().f_code.co_name)


if __name__ == '__main__':
    print("start")
    #指定客户端所在路径
    path = r'D:\huanglong\QuantTrading\国金QMT交易端模拟\bin.x64/../userdata_mini'
    # 生成session id 整数类型 同时运行的策略不能重复
    session_id = int(time.time())
    xt_trader = XtQuantTrader(path, session_id)
    # 开启主动请求接口的专用线程 开启后在on_stock_xxx回调函数里调用XtQuantTrader.query_xxx函数不会卡住回调线程，但是查询和推送的数据在时序上会变得不确定
    # 详见: http://docs.thinktrader.net/vip/pages/ee0e9b/#开启主动请求接口的专用线程
    # xt_trader.set_relaxed_response_order_enabled(True)

    # 创建资金账号为 800068 的证券账号对象
    acc = StockAccount('55004948', 'STOCK')
    # 创建交易回调类对象，并声明接收回调
    callback = MyXtQuantTraderCallback()
    xt_trader.register_callback(callback)
    # 启动交易线程
    xt_trader.start()
    # 建立交易连接，返回0表示连接成功
    connect_result = xt_trader.connect()
    print('建立交易连接，返回0表示连接成功', connect_result)
    # 对交易回调进行订阅，订阅后可以收到交易主推，返回0表示订阅成功
    subscribe_result = xt_trader.subscribe(acc)
    print('对交易回调进行订阅，订阅后可以收到交易主推，返回0表示订阅成功', subscribe_result)

    #这一行是注册全推回调函数 包括下单判断 安全起见处于注释状态 确认理解效果后再放开
    # xtdata.subscribe_whole_quote(["SH", "SZ"], callback=f)
    # 阻塞主线程退出
    #xt_trader.run_forever()
    # 如果使用vscode pycharm等本地编辑器 可以进入交互模式 方便调试 （把上一行的run_forever注释掉 否则不会执行到这里）
    #interact()
    

start
建立交易连接，返回0表示连接成功 0
对交易回调进行订阅，订阅后可以收到交易主推，返回0表示订阅成功 0


***<font color=steel
    size=5
       face=雅黑>
<mark>买入</mark>
</font>***

In [7]:
def get_hold_position():
    # get hold position
    positions = xt_trader.query_stock_positions(account)
    #print("positions:", len(positions))
    #if len(positions) != 0:
    #    print("last position:", positions[-1])
        
    stock_list = []
    position_dict = {}
    for i in range(len(positions)):
        #print("{0} {1} {2} {3} {4} {5}".format(positions[i].account_id, 
        #                           positions[i].stock_code, 
        #                           positions[i].volume, 
        #                           positions[i].open_price, 
        #                           positions[i].market_value, 
        #                           positions[i].avg_price))
        position = {
            "stock_code": positions[i].stock_code, 
            "volume": positions[i].volume, 
            "can_use_volume": positions[i].can_use_volume, 
            "open_price": positions[i].open_price, 
            "market_value": positions[i].market_value, 
            "avg_price": positions[i].avg_price
        }
        stock_list.append(positions[i].stock_code)
        position_dict[positions[i].stock_code] = position
    return stock_list, position_dict
    
bought_stock_list, position_dict = get_hold_position()

***<font color=steel
    size=5
       face=雅黑>
<mark>成交记录读取</mark>
</font>***

In [38]:
current_date = datetime.datetime.now() - datetime.timedelta(days=1)
c_date_str = current_date.strftime("%Y%m%d")
trade_df = pd.read_csv('trade_record_' + c_date_str + '.csv')
trade_list = trade_df['stock_code'].to_list()
trade_list

['600099.SH',
 '600530.SH',
 '600834.SH',
 '600216.SH',
 '000016.SZ',
 '000697.SZ',
 '002278.SZ',
 '300281.SZ',
 '300053.SZ',
 '300152.SZ']

In [26]:
#TODO: 当日已卖出不再买入 √
#TODO: 已持有股票不再买入 √
#TODO: order_target_value √
#TODO: 下跌过程中放量
#TODO：保存成交历史 √
#TODO: 剔除停牌证券 √
current_date = datetime.datetime.now() + datetime.timedelta(days=1)
c_date_str = current_date.strftime("%Y%m%d")
print(c_date_str)
#ret.to_csv('test_data_'+c_date_str+'.csv')
ret = pd.read_csv('test_data_'+c_date_str+'.csv',index_col=0)
######################STRATEGY########################################
# ETF close price may less than 1, otherwise must bigger than 1
stock_df = ret[(ret['va0_va1_5'] >= 4) & (ret['va1_5'] >= 100000)].sort_values(['va0_va1_5', 'va1_5_va5_10'], ascending=False)
######################################################################
stock_list = stock_df.index.to_list()
print(stock_list)


#stock_list = ['588680.SH']

order_value = 100000
for stock in stock_list:
    #TODO：get_market_data_ex
    code = stock.split('.')[0]
    print(code)
    market = stock.split('.')[1]
    
    code_list = [stock]
    openInt = xtdata.get_full_tick(code_list)[stock]['openInt']
    if openInt == 1:
        continue
    
    global context
    context = Context(market, 100000000, c_date_str, c_date_str)
    context.bkt_dt = current_date.strftime("%Y%m%d")
    hist_data = bt.get_hist_data_by_count(context, code, 1, 'none', 'tick', ['lastPrice'])
    #logging.info(hist_data)
    print(hist_data)
    last_price = hist_data.values[0][0]
    print(last_price)
    if last_price <= 0:
        continue
    amount = int(order_value / last_price)
    print(amount)
    if amount % 100 != 0:
        amount = int(amount / 100) * 100
        logging.info('adjusted to buyable amount ' + str(amount))
        
    if stock not in bought_stock_list and stock not in trade_list:
        print("#############actual order: ", stock)
        async_seq = xt_trader.order_stock_async(acc, 
                                                stock, 
                                                xtconstant.STOCK_BUY, 
                                                amount, 
                                                xtconstant.FIX_PRICE, 
                                                last_price, 
                                                'strategy_buy', 
                                                stock)


20240801
['600157.SH', '300061.SZ', '601519.SH', '000750.SZ', '600635.SH', '601698.SH', '159736.SZ', '603883.SH', '300053.SZ', '601059.SH', '513090.SH', '601099.SH', '601881.SH', '002542.SZ', '600011.SH', '600721.SH', '600386.SH', '000980.SZ', '600061.SH', '600609.SH', '512100.SH']
600157
300061
                lastPrice
20240731153000       8.71
8.71
11481
#############actual order:  300061.SZ
601519
                lastPrice
20240731150002       6.13
6.13
16313
#############actual order:  601519.SH
000750
                lastPrice
20240731150000       3.16
3.16
31645
#############actual order:  000750.SZ
600635
                lastPrice
20240731150002       3.89
3.89
25706
#############actual order:  600635.SH
601698
                lastPrice
20240731150003      17.57
17.57
5691
159736
                lastPrice
20240731150006      0.643
0.643
155520
#############actual order:  159736.SZ
603883
                lastPrice
20240731150004      13.89
13.89
7199
#############actual order:  

In [20]:
current_date = datetime.datetime.now() + datetime.timedelta(days=1)
c_date_str = current_date.strftime("%Y%m%d")
c_date_str = '20240731'
print(c_date_str)
#ret.to_csv('test_data_'+c_date_str+'.csv')
ret = pd.read_csv('test_data_'+c_date_str+'.csv',index_col=0)
# ETF close price may less than 1, otherwise must bigger than 1
stock_df = ret[(ret['va0_va1_5'] >= 3) & (ret['va1_5'] >= 100000)].sort_values(['va0_va1_5', 'va1_5_va5_10'], ascending=False)
#ode_list = ['000651.SZ', '600157.SH']
#xtdata.get_full_tick(code_list)
#证券状态
#0,10 - 默认为未知
#1  -  停牌
#11 - 开盘前S
#12 - 集合竞价时段C
#13 - 连续交易T
#14 - 休市B
#15 - 闭市E
#16 - 波动性中断V
#17 - 临时停牌P
#18 - 收盘集合竞价U
#19 - 盘中集合竞价M
#20 - 暂停交易至闭市N
#21 - 获取字段异常
#22 - 盘后固定价格行情
#23 - 盘后固定价格行情完毕
stock_df[['close', 'amount', 'preClose', 'va1_5', 'va5_10', 'va1_5_va5_10', 'va0_va1_5']]


20240731


,close,amount,preClose,va1_5,va5_10,va1_5_va5_10,va0_va1_5
600157.SH,1.10,1.822557e+09,1.120,1829771.8,1591531.4,1.149693,9.593968
002471.SZ,2.19,2.283528e+08,1.990,164494.0,144556.8,1.137919,6.469470
002278.SZ,4.97,4.819315e+08,4.790,168817.4,39890.2,4.232052,5.676133
300053.SZ,10.81,6.812546e+08,9.010,151782.6,128537.6,1.180842,4.466783
600530.SH,3.00,1.396512e+08,2.730,104861.8,44109.6,2.377301,4.461987
600653.SH,1.38,1.351004e+08,1.330,227805.0,148105.6,1.538125,4.285384
000697.SZ,6.88,3.174035e+08,6.330,117329.4,53863.0,2.178293,4.076949
300152.SZ,1.81,1.370627e+08,1.670,186253.4,138315.6,1.346583,3.980040
002561.SZ,8.15,3.323927e+08,7.570,103494.0,155929.8,0.663722,3.976124
000099.SZ,18.03,2.350327e+09,16.390,361919.6,301410.4,1.200754,3.714154


In [16]:
trades = xt_trader.query_stock_trades(account)
#traded_id	str	成交编号
#traded_time	int	成交时间
#traded_price	float	成交均价
#traded_volume	int	成交数量
#traded_amount	float	成交金额
#order_id	int	订单编号
#order_sysid	str	柜台合同编号
#strategy_name	str	策略名称
#order_remark	str	委托备注
#direction

#print(trades)
for i in range(len(trades)):
    print("account_id: ", trades[i].account_id)
    print("stock_code: ", trades[i].stock_code)
    print("order_type: ", trades[i].order_type)
    print("traded_id: ", trades[i].traded_id)
    print("traded_time: ", trades[i].traded_time)
    print("traded_price: ", trades[i].traded_price)
    print("traded_volume: ", trades[i].traded_volume)
    print("traded_amount: ", trades[i].traded_amount)
    print("order_id: ", trades[i].order_id)
    print("strategy_name: ", trades[i].strategy_name)
    print("##################################")



account_id:  55004948
stock_code:  513400.SH
order_type:  24
traded_id:  50009564
traded_time:  1721615045
traded_price:  1.057
traded_volume:  1300
traded_amount:  1374.1
order_id:  1090535588
strategy_name:  strategy_sell
##################################
account_id:  55004948
stock_code:  513730.SH
order_type:  24
traded_id:  50009565
traded_time:  1721615046
traded_price:  1.153
traded_volume:  100
traded_amount:  115.3
order_id:  1090535597
strategy_name:  strategy_sell
##################################
account_id:  55004948
stock_code:  515450.SH
order_type:  24
traded_id:  50009566
traded_time:  1721615047
traded_price:  1.31
traded_volume:  1000
traded_amount:  1310.0
order_id:  1090535604
strategy_name:  strategy_sell
##################################
account_id:  55004948
stock_code:  517520.SH
order_type:  24
traded_id:  50009567
traded_time:  1721615050
traded_price:  1.241
traded_volume:  100
traded_amount:  124.10000000000001
order_id:  1090535615
strategy_name:  strat

***<font color=steel
    size=5
       face=雅黑>
<mark>卖出</mark>
</font>***

In [8]:
#TODO：上涨过程中突然跌量
#TODO: 连续两天成交量是前5天的3倍，提高卖出上线至15%，20%，25%不等
#TODO: 持有5天以上，收益率为正
def check_for_selling(stock_list, position_dict):
    import backtest as bt
    from backtest import Context 
    import pandas as pd
    count = 0
    import datetime
    current_date = datetime.datetime.now() + datetime.timedelta(days=1)
    #current_date = datetime.datetime.strptime('20240621', "%Y%m%d")
    c_date_str = current_date.strftime("%Y%m%d")
    #TODO: get accurate start date
    start_date = current_date
    s_date_str = start_date.strftime("%Y%m%d")
    ret = pd.DataFrame()
    #stock_list = ['517520.SH']
    for key in stock_list:
        if position_dict[key]['can_use_volume'] == 0:
            continue
        logging.info("stock code: " + key)
        code = key.split('.')[0]
        #print(code)
        market = key.split('.')[1]
        
        global context
        context = Context(market, 100000000, s_date_str, c_date_str)
        context.bkt_dt = current_date.strftime("%Y%m%d")
        #TODO：get_market_data_ex
        hist_data = bt.get_hist_data_by_count(context, code, 1, 'front', 'tick', ['lastPrice'])
        #logging.info(hist_data)
        last_price = hist_data.values[0][0]
        logging.info("last_price: " + str(last_price))
        avg_price = position_dict[key]['avg_price']
        logging.info("avg_price: " + str(avg_price))
        
        roe = (last_price / avg_price) - 1
        logging.info("roe: " + str(roe))
        #TODO: 创业板涨跌幅为20%~-10%，其他为10%~-5%
        if roe >= 0.1  or roe <= -0.05:
            logging.warning("sell " + str(position_dict[key]['can_use_volume']) + " shares of " + key)
            async_seq = xt_trader.order_stock_async(acc, 
                                                    key, 
                                                    xtconstant.STOCK_SELL, 
                                                    position_dict[key]['can_use_volume'], 
                                                    xtconstant.FIX_PRICE,
                                                    last_price,
                                                    'strategy_sell'
                                                    )
#xtconstant.FIX_PRICE,
#xtconstant.LATEST_PRICE                                                    last_price,     

In [9]:
def cancel_untraded_order(acc):
    cancelable_order_list = xt_trader.query_stock_orders(acc, cancelable_only = True)
    for cancelable_order in cancelable_order_list:
        cur_time = datetime.datetime.now().timestamp()
        print(cur_time)
        order_time = cancelable_order.order_time
        print(order_time)
        if cur_time - order_time >= 300:
            cancel_result = xt_trader.cancel_order_stock_async(acc, cancelable_order.order_id)

In [ ]:

while True:
    hour = datetime.datetime.now().hour
    minute = datetime.datetime.now().minute
    cur_time = hour * 100 + minute
    logging.info("check time: " + str(cur_time))
    if cur_time < 945 or (cur_time > 1130 and cur_time < 1300):
        logging.info("NOT TRADING TIME")
        time.sleep(60)
        continue
    if  cur_time > 1600:
        break
    cancel_untraded_order(acc)
    stock_list, position_dict = get_hold_position()
    check_for_selling(stock_list, position_dict)
    time.sleep(60)

1722402499.050026
1722396643
2024-07-31 13:08:19.826307 on_cancel_order_stock_async_response
2024-07-31 13:08:19.826307 委托回调 
异步委托回调 
2024-07-31 13:10:18.092936 委托回调 
2024-07-31 13:10:18.092936 委托回调 
1722402686.065235
1722402616
1722402767.879037
1722402616
1722402849.660515
1722402616
1722402932.3042
1722402616
2024-07-31 13:15:32.827018 on_cancel_order_stock_async_response
2024-07-31 13:15:32.827018 委托回调 
异步委托回调 
2024-07-31 13:17:15.437727 委托回调 
1722403101.966032
1722403035
2024-07-31 13:19:33.810657 委托回调 
异步委托回调 
2024-07-31 13:20:02.826610 委托回调 
1722403268.142338
1722403202
1722403354.503024
1722403202
1722403445.44424
1722403202
1722403532.5472
1722403202
2024-07-31 13:25:32.836804 on_cancel_order_stock_async_response
2024-07-31 13:25:32.836804 委托回调 
异步委托回调 
2024-07-31 13:27:19.448215 委托回调 
1722403705.975628
1722403639
1722403790.117698
1722403639
1722403888.594788
1722403639
1722403991.403874
1722403639
2024-07-31 13:33:12.073330 on_cancel_order_stock_async_response
2024-07-31 13:

***<font color=steel
    size=5
       face=雅黑>
<mark>当日成交记录存储</mark>
</font>***

In [36]:
def record_today_trades():
    trades = xt_trader.query_stock_trades(acc)
    col_list = ['account_type',
               'account_id',
               'stock_code',
               'order_type',
               'traded_id',
               'traded_time ',
               'traded_price',
               'traded_volume',
               'traded_amount',
               'order_id',
               'order_sysid ',
               'strategy_name',
               'order_remark',
               'direction',
               'offset_flag']
    
    data = []
    
    for trade in trades:
        col_data = [
                trade.account_type,
                trade.account_id,
                trade.stock_code,
                trade.order_type,
                trade.traded_id,
                trade.traded_time,
                trade.traded_price,
                trade.traded_volume ,
                trade.traded_amount ,
                trade.order_id,
                trade.order_sysid,
                trade.strategy_name ,
                trade.order_remark,
                trade.direction,
                trade.offset_flag
                ]
        data.append(col_data)
    current_date = datetime.datetime.now()
    #current_date = datetime.datetime.strptime('20240621', "%Y%m%d")
    c_date_str = current_date.strftime("%Y%m%d")
    trades_df = pd.DataFrame(data, columns=col_list)
    trades_df.to_csv('trade_record_' + c_date_str + '.csv', index=False)
record_today_trades()